# Using EcoFOCIpy to process raw field data

## DY2504 (Spring Mooring Cruise - Dyson)

## CTD / BTL Data

Basic workflow for each instrument grouping is *(initial archive level)*:
- SBE workflow must happen first
- Parse data from btl files into pandas dataframe

Convert to xarray dataframe for all following work *(working or final data level):
- Add metadata from cruise yaml files and/or header info
- ingest metadata from cruise / cast logs
- process data beyond simple file translate
- apply any calibrations or corrections
    + field corrections
    + offsets
    + instrument compensations
    + some QC were available... this would be old-school simple bounds mostly
- adjust time bounds and sample frequency (xarray dataframe)
- save as CF netcdf via xarray: so many of the steps above are optional
    + **ERDDAP NRT** if no corrections, offsets or time bounds are applied but some meta data is
    + **Working and awaiting QC** has no ERDDAP representation and is a holding spot
    + **ERDDAP Final** fully calibrated, qc'd and populated with meta information

Plot for preview and QC
- preview images (indiv and/or collectively)
- manual qc process
- automated qc process ML/AI

Further refinenments for ERDDAP hosting:

## Differences from EPIC standard
- previously btl files had coordinates of lat/lon/time/depth...with a bottle position / fireing order being a variable, this means that if multiple bottles are fired at the same depth, the file was not uniquely indexed and the bottle variable had to be conflated for multiple fireings
- in new version, we will index based on lat/lon/time/bottle_num (bottle number is a sequential unique value... often representing the position on the rosette), merging with CTD downcast data will require maintining a pressure/depth variable in the bottle data that can be rounded to the nearest 1m bin.  This does not solve the problem with multiple discrete samples taken at a single depth and single niskin though.

## Example below is for SBE 9/11+ V2 but the workflow is similar for any SBE instruments.

Future processing of this instrument can be a simplified (no markdown) process which can be archived so that the procedure can be traced or updated

We process each cast as an individual file so this example will not loop over all profiles.  See `example/all_casts.py` example for processing an entire cruise at once.

Adding Discrete samples such as Oxygen, Chlorophyll, Salinity to BTL Data is in `example/discrete_castdata.py`.  Its purpose is to match niskin/bottle information to depth for the discrete data.

In [1]:
import yaml
import glob

import EcoFOCIpy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import EcoFOCIpy.io.ncCFsave as ncCFsave
import EcoFOCIpy.metaconfig.load_config as load_config

The sample_data_dir should be included in the github package but may not be included in the pip install of the package

## Simple Processing - first step

In [2]:
sample_data_dir = '/Users/bell/ecoraid/2025/CTDcasts/dy2504/' #root path to cruise directory
ecofocipy_dir = '/Users/bell/Programs/EcoFOCIpy/'

In [3]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'rawconverted/' #<- point to cruise and process all files within
cruise_name = 'DY2504' #no hyphens
cruise_meta_file = sample_data_dir+'logs/DY2504.yaml'
inst_meta_file = sample_data_dir+'logs/FOCI_standard_CTD.yaml' #<- copy to each deployment for simplicity?
group_meta_file = ecofocipy_dir+'staticdata/institutional_meta_example.yaml'
###############################################################

#init and load data
cruise = sbe_ctd_parser.sbe_btl()
filename_list = sorted(glob.glob(datafile + '*.btl'))

cruise_data = cruise.manual_parse(filename_list)

Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD001.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD002.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD003.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD004.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD005.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD006.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD008.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD009.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD010.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD011.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD012.btl


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD013.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD014.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD015.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD016.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD017.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD018.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD019.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD020.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD021.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD022.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD023.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD024.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD025.btl


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD026.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD027.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD028.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD029.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD030.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD031.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD032.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD033.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD034.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD035.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD036.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD037.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD038.btl


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD039.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD040.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD041.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD042.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD043.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD044.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD045.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD046.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD050.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD051.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD052.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD053.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD054.btl
Processing /

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD060.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD061.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD062.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD063.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2504/rawconverted/CTD064.btl


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])


In [4]:
#quick statistical look at the distribution of data for a cast
# #preview a dataframe
cruise_data['CTD001.btl'].describe()

,sal11,sal00,sbeox0ml/l,sbeox0ps,sbox0mm/kg,sbeox1ml/l,sbeox1ps,sbox1mm/kg,sigma-t00,sigma-t11,c0ms/cm,c1ms/cm,fleco-afl,t090c,t190c,turbwetntu0,par,prdm,scan,datetime
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5
mean,32.420820,32.421340,6.490560,90.693600,282.631200,6.547620,91.491000,285.115800,25.605540,25.605100,31.461046,31.460709,0.105620,5.231380,5.231520,1.188980,0.001714,95.186600,7034.000000,2025-04-26 10:05:33.600000
min,32.420600,32.421100,6.485300,90.619000,282.401000,6.539900,91.382000,284.778000,25.605400,25.604900,31.460651,31.460380,0.097400,5.231200,5.231400,1.163700,0.000201,95.030000,6955.000000,2025-04-26 10:05:30
25%,32.420800,32.421200,6.489300,90.676000,282.578000,6.545300,91.460000,285.017000,25.605400,25.605100,31.460670,31.460485,0.104400,5.231200,5.231400,1.184600,0.000928,95.070000,6999.000000,2025-04-26 10:05:32
50%,32.420800,32.421400,6.491000,90.700000,282.652000,6.546400,91.474000,285.065000,25.605600,25.605100,31.461015,31.460582,0.105400,5.231300,5.231400,1.189800,0.001487,95.133000,7039.000000,2025-04-26 10:05:34
75%,32.420900,32.421500,6.492800,90.725000,282.727000,6.550200,91.527000,285.227000,25.605600,25.605200,31.461439,31.460856,0.109800,5.231600,5.231500,1.197100,0.001645,95.323000,7071.000000,2025-04-26 10:05:35
max,32.421000,32.421500,6.494400,90.748000,282.798000,6.556300,91.612000,285.492000,25.605700,25.605200,31.461457,31.461241,0.111100,5.231600,5.231900,1.209700,0.004307,95.377000,7106.000000,2025-04-26 10:05:37
std,0.000148,0.000182,0.003508,0.049652,0.152688,0.006092,0.085247,0.264737,0.000134,0.000122,0.000394,0.000346,0.005399,0.000205,0.000217,0.016987,0.001556,0.154798,59.253692,NaN


## Create BTL report file

In [5]:
# btl report file
for cast in cruise_data.keys():
    try:
        df=cruise_data[cast]
        df['cast'] = cast.lower().split('.')[0]
        if cast.lower().split('.')[0] == 'ctd001':
            df.to_csv(f'{cruise_name}.report_btl')
        else:
            df.to_csv(f'{cruise_name}.report_btl',mode='a', header=False)
    except:
        print(f'some issue in {cast}')

## Time Properties

Not traditionally dealt with for CTD files as they are likely dynamically updated via GPS feed.  However, FOCI tends to label the date/time with the ***at depth*** time-stamp

## Depth Properties and other assumptions

- currently, all processing and binning (1m for FOCI) is done via seabird routines and the windows software.  This may change with the python ctd package for a few tasks

## Add Deployment meta information

In [6]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)
cruise_config[cruise_name]

{'CruiseID': 'DY2504',
 'CruiseID_Historic': '',
 'CruiseID_Alternates': '',
 'Project_Leg': '',
 'Vessel': 'NOAAS Oscar Dyson',
 'ShipID': 'DY',
 'StartDate': datetime.date(2025, 4, 26),
 'EndDate': datetime.date(2025, 5, 12),
 'Project': 'Spring Mooring Cruise',
 'ChiefScientist': 'Shaun Bell',
 'StartPort': 'Kodiak, AK',
 'EndPort': 'Dutch Harbor, AK',
 'CruiseLocation': 'Bering Sea',
 'Description': 'FOCI Spring Mooring Survey',
 'CruiseYear': 2025,
 'ctdlogs_pdf_name': 'DY2504_CTDCastLogs.pdf',
 'Accession': '',
 'Links': '\r',
 'QC_Status': '',
 'QC_Signature': ''}

In [7]:
#and if you want a cast from the cruise, just use the consective cast number
cruise_config['CTDCasts']['CTD001']

{'id': 54956,
 'Vessel': 'NOAAS Oscar Dyson',
 'CruiseID': 'DY2504',
 'Project_Leg': '',
 'UniqueCruiseID': 'DY2504',
 'Project': 'Spring Mooring Cruise',
 'StationNo_altname': '',
 'ConsecutiveCastNo': 'CTD001',
 'LatitudeDeg': 54,
 'LatitudeMin': 4.91,
 'LongitudeDeg': 163,
 'LongitudeMin': 29.92,
 'GMTDay': 26,
 'GMTMonth': 'Apr',
 'GMTYear': 2025,
 'GMTTime': 65100,
 'DryBulb': 5.0,
 'RelativeHumidity': 94,
 'WetBulb': -99.9,
 'Pressure': 998,
 'SeaState': '',
 'Visibility': '',
 'WindDir': 268,
 'WindSpd': 12.0,
 'CloudAmt': '',
 'CloudType': '',
 'Weather': '',
 'SurfaceTemp': -99.9,
 'BottomDepth': 99,
 'StationNameID': 'UN04',
 'MaxDepth': 95,
 'InstrumentSerialNos': '',
 'Notes': 'deploy AL25_AU_UN04 (mml) - test bottles',
 'NutrientBtlNiskinNo': '1',
 'NutrientBtlNumbers': '',
 'OxygenBtlNiskinNo': '1',
 'OxygenBtlNumbers': '221',
 'SalinityBtlNiskinNo': '',
 'SalinityBtlNumbers': '',
 'ChlorophyllBtlNiskinNo': '1',
 'ChlorophyllBtlVolumes': '',
 'InstrumentType': 'SBE911plus

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [8]:
with open(inst_meta_file) as file:
    inst_config = yaml.full_load(file)
inst_config

{'time': {'epic_key': 'TIM_601',
  'name': 'time',
  'generic_name': 'time',
  'standard_name': 'time',
  'long_name': 'date and time since reference time'},
 'depth': {'epic_key': 'D_3',
  'generic_name': 'depth',
  'units': 'meter',
  'long_name': 'depth below surface (meters)',
  'standard_name': 'depth',
  'positive': 'down'},
 'latitude': {'epic_key': 'LON_501',
  'name': 'latitude',
  'generic_name': 'latitude',
  'units': 'degrees_north',
  'long_name': 'latitude',
  'standard_name': 'latitude'},
 'longitude': {'epic_key': 'LAT_500',
  'name': 'longitude',
  'generic_name': 'longitude',
  'units': 'degrees_east',
  'long_name': 'longitude',
  'standard_name': 'longitude'},
 'temperature_ch1': {'epic_key': 'T_28',
  'generic_name': 'temp channel 1',
  'long_name': 'Sea temperature in-situ ITS-90 scale',
  'standard_name': 'sea_water_temperature',
  'units': 'degree_C'},
 'temperature_ch2': {'epic_key': 'T2_35',
  'generic_name': 'temp channel 2',
  'long_name': 'Sea temperature i

In [9]:
#sbe data uses header info to name variables... but we want standard names from the dictionary I've created, so we need to rename column variables appropriately
#rename values to appropriate names, if a value isn't in the .yaml file, you can add it

#*** biggest *** difference between moored and profile data is there may be multiple instruments with the same dataype (e.g.) temperature
# on the same platform.  We _used_ to use the phrases primary and secondary, but will now only refer to them as ch1, ch2 etc
cruise_data['CTD001.btl'] = cruise_data['CTD001.btl'].rename(columns={
                        't090c':'temperature_ch1',
                        't190c':'temperature_ch2',
                        'sal00':'salinity_ch1',
                        'sal11':'salinity_ch2',
                        'sbox0mm/kg':'oxy_conc_ch1',
                        'sbeox0ml/l':'oxy_concM_ch1',
                        'sbox1mm/kg':'oxy_conc_ch2',
                        'sbeox1ml/l':'oxy_concM_ch2',
                        'sbeox0ps':'oxy_percentsat_ch1',
                        'sbeox1ps':'oxy_percentsat_ch2',
                        'sigma-t00':'sigma_t_ch1',
                        'sigma-t11':'sigma_t_ch2',
                        'cstarat0':'Attenuation',
                        'cstartr0':'Transmittance',
                        'fleco-afl':'chlor_fluorescence',
                        'turbwetntu0':'turbidity',
                        'empty':'empty', #this will be ignored
                        'prdm':'Pressure [dbar]',
                        'flag':'flag'})

cruise_data['CTD001.btl'].sample()

,salinity_ch2,salinity_ch1,oxy_concM_ch1,oxy_percentsat_ch1,oxy_conc_ch1,oxy_concM_ch2,oxy_percentsat_ch2,oxy_conc_ch2,sigma_t_ch1,sigma_t_ch2,...,c1ms/cm,chlor_fluorescence,temperature_ch1,temperature_ch2,turbidity,par,Pressure [dbar],scan,datetime,cast
bottle,,,,,,,,,,,,,,,,,,,,,
4.0,32.4208,32.4212,6.4853,90.619,282.401,6.5399,91.382,284.778,25.6054,25.6051,...,31.460485,0.1098,5.2312,5.2314,1.1971,0.000201,95.03,7071.0,2025-04-26 10:05:35,ctd001


In [10]:
cruise_data['CTD001.btl'].columns

Index(['salinity_ch2', 'salinity_ch1', 'oxy_concM_ch1', 'oxy_percentsat_ch1',
       'oxy_conc_ch1', 'oxy_concM_ch2', 'oxy_percentsat_ch2', 'oxy_conc_ch2',
       'sigma_t_ch1', 'sigma_t_ch2', 'c0ms/cm', 'c1ms/cm',
       'chlor_fluorescence', 'temperature_ch1', 'temperature_ch2', 'turbidity',
       'par', 'Pressure [dbar]', 'scan', 'datetime', 'cast'],
      dtype='object')

## Add institutional meta-information


In [11]:
with open(group_meta_file) as file:
    group_config = yaml.full_load(file)
group_config

{'source_documents': 'http://www.oceansites.org/docs/oceansites_data_format_reference_manual.pdf',
 'institution': 'Pacific Marine Environmental Lab (PMEL)',
 'project': 'EcoFOCI',
 'project_url': 'https://www.ecofoci.noaa.gov',
 'principal_investigator': 'Phyllis Stabeno',
 'principal_investigator_email': 'phyllis.stabeno (at) noaa.gov',
 'creator_name': 'Shaun Bell',
 'creator_email': 'shaun.bell (at) noaa.gov',
 'creator_institution': 'PMEL',
 'keywords': 'Mooring, Oceanographic',
 'comment': 'Provisional data',
 'sea_area': 'Bering Sea (BS)',
 'featureType': 'timeSeries',
 'conventions': '”CF-1.6, ~OceanSITES-1.5, ACDD-1.2”',
 'license': '',
 'references': '',
 'citation': '',
 'acknowledgement': ''}

In [12]:
# Add meta data and prelim processing based on meta data
# Convert to xarray and add meta information - save as CF netcdf file
# pass -> data, instmeta, depmeta
cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data['CTD001.btl'], 
                                instrument_yaml=inst_config, 
                                operation_yaml=cruise_config,
                                operation_type='ctd')
cruise_data_nc

In [13]:
cruise_data_nc.get_xdf()

<xarray.Dataset> Size: 880B
Dimensions:             (bottle: 5)
Coordinates:
  * bottle              (bottle) float64 40B 1.0 2.0 3.0 4.0 5.0
Data variables: (12/21)
    salinity_ch2        (bottle) float64 40B 32.42 32.42 32.42 32.42 32.42
    salinity_ch1        (bottle) float64 40B 32.42 32.42 32.42 32.42 32.42
    oxy_concM_ch1       (bottle) float64 40B 6.494 6.493 6.491 6.485 6.489
    oxy_percentsat_ch1  (bottle) float64 40B 90.75 90.72 90.7 90.62 90.68
    oxy_conc_ch1        (bottle) float64 40B 282.8 282.7 282.7 282.4 282.6
    oxy_concM_ch2       (bottle) float64 40B 6.545 6.556 6.55 6.54 6.546
    ...                  ...
    turbidity           (bottle) float64 40B 1.164 1.19 1.21 1.197 1.185
    par                 (bottle) float64 40B 0.001645 0.0009277 ... 0.004307
    Pressure [dbar]     (bottle) float64 40B 95.38 95.32 95.13 95.03 95.07
    scan                (bottle) float64 40B 6.955e+03 6.999e+03 ... 7.106e+03
    datetime            (bottle) datetime64[ns] 40B 2025-04-26T10:05:30 ... 2...
    cast                (bottle) object 40B 'ctd001' 'ctd001' ... 'ctd001'

At this point, you could save your file with the `.xarray2netcdf_save()` method and have a functioning dataset.... but it would be very simple with no additional qc, meta-data, or tuned parameters for optimizing software like ferret or erddap.

In [14]:
# expand the dimensions and coordinate variables
# renames them appropriatley and prepares them for meta-filled values
cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

In [15]:
#build list from columsn in data - if a variable isn't in the yaml file, it will be dropped from the final data fields
cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data['CTD001.btl'].columns.values),drop_missing=False)
#adding dimension meta needs to come after updating the dimension values... BUG?
cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])

The following steps can happen in just about any order and are all meta-data driven.  Therefore, they are not required to have a functioning dataset, but they are required to have a well described dataset

In [16]:
cruise_data_nc.get_xdf()

<xarray.Dataset> Size: 904B
Dimensions:             (latitude: 1, longitude: 1, time: 1, bottle: 5)
Coordinates:
  * bottle              (bottle) float64 40B 1.0 2.0 3.0 4.0 5.0
  * latitude            (latitude) float64 8B 1e+35
  * longitude           (longitude) float64 8B 1e+35
  * time                (time) float64 8B 1e+35
Data variables: (12/21)
    salinity_ch2        (latitude, longitude, time, bottle) float64 40B 32.42...
    salinity_ch1        (latitude, longitude, time, bottle) float64 40B 32.42...
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 40B 6.494...
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 40B 90.75...
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 40B 282.8...
    oxy_concM_ch2       (latitude, longitude, time, bottle) float64 40B 6.545...
    ...                  ...
    turbidity           (latitude, longitude, time, bottle) float64 40B 1.164...
    par                 (latitude, longitude, time, bottle) float64 40B 0.001...
    Pressure [dbar]     (latitude, longitude, time, bottle) float64 40B 95.38...
    scan                (latitude, longitude, time, bottle) float64 40B 6.955...
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 40B ...
    cast                (latitude, longitude, time, bottle) object 40B 'ctd00...

In [17]:
#add global attributes
cruise_data_nc.deployment_meta_add(conscastno='CTD001')

#add instituitonal global attributes
cruise_data_nc.institution_meta_add(group_config)

#add creation date/time - provenance data
cruise_data_nc.provinance_meta_add()

#provide intial qc status field
cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

cruise_data_nc.get_xdf()

<xarray.Dataset> Size: 904B
Dimensions:             (latitude: 1, longitude: 1, time: 1, bottle: 5)
Coordinates:
  * bottle              (bottle) float64 40B 1.0 2.0 3.0 4.0 5.0
  * latitude            (latitude) float64 8B 1e+35
  * longitude           (longitude) float64 8B 1e+35
  * time                (time) float64 8B 1e+35
Data variables: (12/21)
    salinity_ch2        (latitude, longitude, time, bottle) float64 40B 32.42...
    salinity_ch1        (latitude, longitude, time, bottle) float64 40B 32.42...
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 40B 6.494...
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 40B 90.75...
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 40B 282.8...
    oxy_concM_ch2       (latitude, longitude, time, bottle) float64 40B 6.545...
    ...                  ...
    turbidity           (latitude, longitude, time, bottle) float64 40B 1.164...
    par                 (latitude, longitude, time, bottle) float64 40B 0.001...
    Pressure [dbar]     (latitude, longitude, time, bottle) float64 40B 95.38...
    scan                (latitude, longitude, time, bottle) float64 40B 6.955...
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 40B ...
    cast                (latitude, longitude, time, bottle) object 40B 'ctd00...
Attributes: (12/48)
    CruiseID:                      DY2504
    CruiseID_Historic:             
    CruiseID_Alternates:           
    Project_Leg:                   
    Vessel:                        NOAAS Oscar Dyson
    ShipID:                        DY
    ...                            ...
    references:                    
    citation:                      
    acknowledgement:               
    date_created:                  2025-05-20T07:14:16Z
    date_modified:                 
    QC_indicator:                  unqcd

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [18]:
#loop over all casts and perform tasks shown above

for cast in cruise_data.keys():
    try:
        cruise_data[cast] = cruise_data[cast].rename(columns={
                            't090c':'temperature_ch1',
                            't190c':'temperature_ch2',
                            'sal00':'salinity_ch1',
                            'sal11':'salinity_ch2',
                            'sbox0mm/kg':'oxy_conc_ch1',
                            'sbeox0ml/l':'oxy_concM_ch1',
                            'sbox1mm/kg':'oxy_conc_ch2',
                            'sbeox1ml/l':'oxy_concM_ch2',
                            'sbeox0ps':'oxy_percentsat_ch1',
                            'sbeox1ps':'oxy_percentsat_ch2',
                            'sigma-t00':'sigma_t_ch1',
                            'sigma-t11':'sigma_t_ch2',
                            'cstarat0':'Attenuation',
                            'cstartr0':'Transmittance',
                            'fleco-afl':'chlor_fluorescence',
                            'turbwetntu0':'turbidity',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

        cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                                    instrument_yaml=inst_config, 
                                    operation_yaml=cruise_config,
                                    operation_type='ctd')

        cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

        cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=True)
        #adding dimension meta needs to come after updating the dimension values... BUG?
        cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
        cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

        #add global attributes
        cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

        #add instituitonal global attributes
        cruise_data_nc.institution_meta_add(group_config)

        #add creation date/time - provenance data
        cruise_data_nc.provinance_meta_add()

        #provide intial qc status field
        cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

        cast_label = cast.lower().split('d')[-1].split('.')[0]
        cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                                   filename=cruise_name+'c'+cast_label.zfill(3)+'_btl.nc',format="NETCDF3_CLASSIC")

  
    except:
        print(f'Skipping {cast}')

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'minutes' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'minutes' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncC

## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history